In [1]:
%cd 'Q:\Dropbox\python\QSim\'

Q:\Dropbox\python\QSim


In [2]:
%matplotlib qt
from scipy.io import *
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
import numpy as np
#from numpy import fft
#from numpy import linalg
#from scipy import interpolate
#from sklearn.cluster import DBSCAN
#from scipy import optimize
from pyfield import *
import time

In [3]:
s = np.load('data\sdo.x766y1118.20110225.npz')

data = s['hmi'][129,:,:]
#data[np.where(np.abs(data) < 10)] = 0
#fig = plt.figure(figsize=(8,8))
#plt.show()

In [4]:
field = Field(data,1.).search_charges(25).search_nulls()

Searching charges...
Done! Elapsed time: 0.00599980354309
Searching nulls...
Done! Elapsed time: 0.891000032425


In [5]:
field.draw_footprint()

<pyfield.Field instance at 0x0000000015DB7B88>

In [5]:
field.connectivity()

array([[303, 179, 123,   0,  11,  24,  19,   0],
       [  0,  17,   0,   0,  18,   0,  38,   0],
       [  0,   0, 358, 901, 171, 112,  14, 273],
       [965,   0, 141,  83,   0, 342,   0,   4]])

In [54]:
def search_nulls(self, maxi = 10, h = 0.1, d = 2):
                    
    xi = np.arange(0,self.dim[0]/d)*d
    yi = np.arange(0,self.dim[1]/d)*d

    xx, yy = np.meshgrid(xi, yi)
    zz = np.zeros(self.dim[0]*self.dim[1]/d**2)

    xx = xx.flatten()
    yy = yy.flatten()

    B = np.zeros((2,2,self.dim[0]*self.dim[1]/d**2))
    
    """
    self.Update(np.array([xx+h, yy, zz])).Get().Jacobian()
    hesx = np.sum(self._jacobian*self._p, axis = 1)
        
    self.Update(np.array([xx-h, yy, zz])).Get().Jacobian()
    hesx = (hesx - np.sum(self._jacobian*self._p, axis = 1))/(2*h)
    
    self.Update(np.array([xx, yy+h, zz])).Get().Jacobian()
    hesy = np.sum(self._jacobian*self._p, axis = 1)
        
    self.Update(np.array([xx, yy-h, zz])).Get().Jacobian()
    hesy = (hesy - np.sum(self._jacobian*self._p, axis = 1))/(2*h)
    
    det = hesx[0,:]*hesy[1,:] - hesx[1,:]*hesy[0,:]
    
    B[0,0,:] = hesy[1,:]/det
    B[1,1,:] = hesx[0,:]/det
    B[0,1,:] = -hesy[0,:]/det
    B[1,0,:] = -hesx[1,:]/det
    """
    B[0,0,:] += 1
    B[1,1,:] += 1
    
    self.Update(np.array([xx, yy, zz])).Get().Jacobian()
    gr = np.sum(self._jacobian*self._p, axis = 1)[0:2,:]
    
    for i in range(0,50):
        sk = - np.sum(B*gr, axis = 1)*0.1
        xx = xx + sk[0,:]
        yy = yy + sk[1,:]

        self.Update(np.array([xx, yy, zz])).Get().Jacobian()
        _gr = gr
        gr = np.sum(self._jacobian*self._p, axis = 1)[0:2,:]    
        yk = gr - _gr

        B = (B + (np.sum(sk*yk, axis = 0) + np.sum(np.sum(B*yk, axis = 1)*yk, axis = 0))/np.sum(sk*yk, axis = 0)**2*
               np.array([[sk[i,:]*sk[j,:] for i in range(0,2)] for j in range(0,2)]) - 
               (np.array([[np.sum(B*yk, axis = 1)[j,:]*sk[i,:] for i in range(0,2)] for j in range(0,2)]) + 
                np.sum(B*np.array([[sk[j,:]*yk[i,:] for i in range(0,2)] for j in range(0,2)]), axis = 0))/np.sum(sk*yk, axis = 0))
             
    
    
    return xx, yy, zz

xx, yy, zz = search_nulls(field)



In [55]:
field.draw_footprint()
plt.axis([0,64,0,64])
plt.plot(xx, yy, 'ro')

In [165]:
b = B[0,0,:]
b.shape = (32,32)
plt.imshow(b)

In [14]:
q = FField(data,1)

det = q[3]*q[5]-q[4]**2

dx = -(q[1]*q[5]-q[2]*q[4])/det
dy = -(q[2]*q[3]-q[1]*q[4])/det

q1 = q[0]+q[1]*dx+q[2]*dy+q[3]*dx**2/2+q[4]*dx*dy+q[5]*dy**2

#dx = -q[1]
#dy = -q[2]


In [18]:
t = np.where((np.abs(dx) < 1)*(np.abs(dy) < 1)*(np.abs(q1) > 30)*(det > 0))
plt.imshow(q[0])
plt.plot(t[1]+dx[t],t[0]+dy[t], 'ro')

In [63]:
plt.imshow(s['aia_171'][503,:,:])

In [78]:
plt.imshow(s['aia_171'][503,:,:])

In [7]:
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp1d
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for null in [field.Nulls[3], field.Nulls[8]]:
    vecs = null.eigen[1][:,np.where(np.sign(null.eigen[0]) == -np.sign(null.det))[0]]

    verts = []

    for i in range(0,51):
        phi = np.pi*i/50
        dx = vecs[:,0]*np.cos(phi)+vecs[:,1]*np.sin(phi)
        v = np.column_stack((null.x, field.rk2(null.x + dx, mul = -np.sign(null.det))[0]))
        v1 = []
        for vi in v:
            n = np.shape(vi)[0]
            t = np.arange(0,n)
            t1 = np.arange(0,11)/10.*(n-1)
            v1.extend([interp1d(t,vi)(t1)])
        ax.plot(v1[0], v1[1], v1[2], color = 'green')
        verts.extend([np.array(v1)])

    verts = np.array(verts)

    #ax.plot_wireframe(verts[:,0,:],verts[:,1,:],verts[:,2,:])

#plt.axis([0,64,0,64])
plt.show()

C:\Anaconda\lib\site-packages\mpl_toolkits\mplot3d\axes3d.py:1094: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.button_pressed in self._rotate_btn:
